# Advanced Prompting avec OpenAI

Dans ce notebook, nous allons tester différentes techniques avancées de **prompt engineering**:
- **Zero-shot prompting**
- **Few-shot prompting**
- **Chain-of-thought** (CoT)
- **Self-refine** (ou auto-amélioration)

Nous utiliserons la **nouvelle API** de la bibliothèque `openai` (>=1.0.0) via la classe `OpenAI` et ses méthodes de chat (`client.chat.completions.create`).


In [15]:
# ============================
# Cellule 1 : Installation
# ============================

%pip install openai tiktoken python-dotenv
# Remarque : Aucune fin de ligne en commentaire pour éviter l'erreur


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
# ============================
# Cellule 2 : Configuration
# ============================

import os
from dotenv import load_dotenv

load_dotenv()

# On suppose que ton .env contient :
# OPENAI_API_KEY=sk-xxxxxx
# (ou autre variable si tu utilises Azure)
#
# Récupère la clé d'API
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Clé API introuvable. Vérifie ton fichier .env.")


In [17]:
# ============================
# Cellule 3 : Client OpenAI
# ============================

import openai
from openai import OpenAI

# Pour l'exemple, on définit le modèle par défaut
MODEL_NAME = "gpt-4o-mini"

# Instanciation du client
client = OpenAI(
    api_key=api_key,
    # Tu peux configurer d'autres options si besoin
)

print("Client OpenAI initialisé avec succès !")


Client OpenAI initialisé avec succès !


# Rappels sur le Prompt Engineering avancé

Plusieurs techniques de `prompt engineering` permettent d'améliorer les réponses d'un LLM:

1. **Zero-shot prompting**  
   On formule une simple question (ou instruction), sans donner d'exemples, et le modèle déduit la tâche.

2. **Few-shot prompting**  
   On fournit quelques exemples (input → output) pour guider le modèle vers le style, le format ou la logique attendus.

3. **Chain-of-thought (CoT)**  
   On incite le modèle à décomposer sa réflexion par étapes. Cela améliore souvent la justesse sur des problèmes complexes (calculs, raisonnement).

4. **Self-refine**  
   On demande au modèle de s'auto-corriger (self-critique) puis de proposer une version améliorée de sa réponse.

Nous allons montrer quelques brefs exemples !


In [18]:
# ============================
# Cellule 5 : Zero-shot
# ============================

prompt_1 = "Donne-moi 3 idées de recettes végétariennes à base de tomates."
response_1 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt_1}
    ],
    # Contrôle du style
    max_tokens=200,
    temperature=0.7  # plus la température est haute, plus c'est créatif
)

print("=== Zero-shot Prompt ===")
print(f"Prompt: {prompt_1}\n")
print("Réponse du modèle :\n")
print(response_1.choices[0].message.content)


=== Zero-shot Prompt ===
Prompt: Donne-moi 3 idées de recettes végétariennes à base de tomates.

Réponse du modèle :

Bien sûr ! Voici trois idées de recettes végétariennes à base de tomates :

### 1. **Tarte à la tomate et au fromage de chèvre**
**Ingrédients :**
- Pâte brisée
- Tomates fraîches (variété de votre choix)
- Fromage de chèvre
- Herbes de Provence
- Huile d'olive
- Sel et poivre

**Instructions :**
1. Préchauffez le four à 180°C.
2. Étalez la pâte brisée dans un moule à tarte et piquez le fond avec une fourchette.
3. Coupez les tomates en rondelles et disposez-les sur la pâte.
4. Émiettez le fromage de chèvre par-dessus.
5. Arrosez d'un filet d'huile d'olive, saupoudrez d'herbes de Provence, de sel et de poivre.
6. Enfournez pendant 30 minutes ou jusqu


Ici, pas d’exemples ni d’instructions détaillées, on se contente d’un prompt direct.



###  Exemple Few-shot prompting (Code)

In [19]:
# ============================
# Cellule 6 : Few-shot
# ============================

few_shot_prompt = """
Tu es un assistant culinaire spécialisé en recettes végétariennes.
Voici des exemples :

Exemple 1:
Q: Quelles idées de salade d'été me proposes-tu ?
A: - Salade de quinoa et tomates cerises
   - Salade de lentilles aux oignons rouges
   - ...

Exemple 2:
Q: Je veux cuisiner des champignons, as-tu une idée ?
A: - Poêlée de champignons, ail et persil
   - ...

Maintenant, voici ma question:

Q: Propose-moi 2 plats végétariens sans gluten, si possible avec des tomates.
A:
"""

response_2 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": few_shot_prompt}
    ],
    max_tokens=200,
    temperature=0.7
)

print("=== Few-shot Prompt ===")
print("Réponse du modèle :\n")
print(response_2.choices[0].message.content)


=== Few-shot Prompt ===
Réponse du modèle :

- Ratatouille aux légumes du soleil : un mélange savoureux d'aubergines, de courgettes, de poivrons et de tomates, mijoté avec des herbes de Provence.
  
- Tomates farcies aux quinoa et légumes : des tomates creusées et remplies d'un mélange de quinoa, de courgettes, d'oignons et d'épices, puis cuites au four.


Ici, nous donnons au modèle **deux exemples** de questions/réponses avant la **véritable question**. Cela oriente le style et le contexte.



###  7 : Exemple Chain-of-thought (Code)

On va demander un **calcul** simple, en guidant le modèle à réfléchir pas à pas :

In [20]:
# ============================
# Cellule 7 : Chain-of-thought
# ============================

cot_prompt = """
Alice a 5 pommes, elle en jette 2, puis elle en donne 1 à Bob.
Bob lui rend ensuite 1 pomme.
Combien de pommes Alice a-t-elle à la fin ?

Explique ton raisonnement étape par étape, puis donne la réponse finale.
"""

response_3 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": cot_prompt}
    ],
    max_tokens=200,
    temperature=0.2  # on réduit la température pour moins de fantaisie
)

print("=== Chain-of-thought Prompt ===")
print("Réponse du modèle (avec raisonnement) :\n")
print(response_3.choices[0].message.content)


=== Chain-of-thought Prompt ===
Réponse du modèle (avec raisonnement) :

Pour résoudre ce problème, nous allons suivre les étapes une par une.

1. **Alice commence avec 5 pommes.**
   - Pommes d'Alice : 5

2. **Elle en jette 2.**
   - Pommes d'Alice après avoir jeté 2 : 5 - 2 = 3
   - Pommes d'Alice : 3

3. **Elle en donne 1 à Bob.**
   - Pommes d'Alice après avoir donné 1 à Bob : 3 - 1 = 2
   - Pommes d'Alice : 2

4. **Bob lui rend ensuite 1 pomme.**
   - Pommes d'Alice après que Bob lui a rendu 1 : 2 + 1 = 3
   - Pommes d'Alice : 3

En résumé, après toutes ces actions, Alice a 3 pommes à la fin.

**Réponse finale : Alice a


On demande explicitement « explique ton raisonnement ». Cela **n’oblige** pas le modèle à le faire, mais en pratique, GPT-4o-mini (ou tout modèle qui gère le CoT) fournit souvent une solution pas-à-pas.

---

###  8 : Exemple Self-refine (Code)

L’idée : on fait **une première demande** (première réponse) et ensuite **on redemande** au modèle de s’auto-corriger.

#### 8a. Premier prompt

In [21]:
# ============================
# Cellule 8a : Self-refine Step 1
# ============================

prompt_sr1 = """
Ecris une courte fonction Python pour calculer la somme d'une liste. 
Ajoute un bug volontaire dans le code. 
"""

response_sr1 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": prompt_sr1}],
    max_tokens=300
)

buggy_code = response_sr1.choices[0].message.content

print("=== Self-refine (1) : Code buggy ===\n")
print(buggy_code)


=== Self-refine (1) : Code buggy ===

Voici une courte fonction Python qui calcule la somme d'une liste, avec un bug volontaire inclus :

```python
def somme_liste(lst):
    total = 0
    for i in range(len(lst)):
        total += lst[i]  # Bug : utilisation de 'range(len(lst))' au lieu de 'for i in lst'
    return total

# Exemple d’utilisation
ma_liste = [1, 2, 3, 4, 5]
print(somme_liste(ma_liste))  # Cela ne donnera pas la somme correcte
```

Dans cette version, le bug est que nous utilisons `range(len(lst))` pour accéder aux éléments de la liste au lieu d'itérer directement sur les éléments de la liste avec `for i in lst`.


#### 8b. Self-critique et amélioration

In [22]:
# ============================
# Cellule 8b : Self-refine Step 2
# ============================

prompt_sr2 = f"""
Voici un code Python qui contient un bug:

{buggy_code}

Peux-tu l'analyser, détecter le bug, proposer un correctif et une version améliorée du code ? 
Explique la correction.
"""

response_sr2 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": prompt_sr2}],
    max_tokens=400,
    temperature=0.3
)

print("=== Self-refine (2) : Correction ===\n")
print(response_sr2.choices[0].message.content)


=== Self-refine (2) : Correction ===

Analysons le code Python fourni et identifions le bug.

### Analyse du code

La fonction `somme_liste` est censée calculer la somme des éléments d'une liste. Le code utilise une boucle `for` avec `range(len(lst))`, ce qui signifie qu'il itère sur les indices de la liste plutôt que sur les éléments eux-mêmes. Cependant, dans ce cas, le code fonctionne correctement et renvoie la somme des éléments de la liste, car `lst[i]` accède bien aux éléments de la liste en utilisant les indices.

### Bug identifié

Le bug mentionné dans le code n'est pas un bug fonctionnel, car le code renvoie effectivement la somme correcte. Cependant, il est vrai que l'utilisation de `range(len(lst))` est moins idiomatique en Python. La manière plus pythonique d'itérer sur les éléments d'une liste est d'utiliser `for i in lst`, ce qui rend le code plus lisible et plus concis.

### Correctif et version améliorée

Voici la version corrigée et améliorée du code :

```python
def 

Ici, on utilise la première réponse pour nourrir le second prompt, demandant au modèle de **critiquer** et **améliorer** la réponse initiale.

---

### Cellule 9 : Interactive Prompt (Code)

Enfin, on peut proposer une **cellule interactive** : l’utilisateur peut saisir un prompt, et on envoie la requête au modèle :

In [23]:
# ============================
# Cellule 9 : Prompt interactif
# ============================

while True:
    user_input = input("Tape ton prompt ('exit' pour quitter) : ")
    if user_input.strip().lower() in ["exit", "quit"]:
        print("Fin de l'interaction.")
        break
    
    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role":"user","content":user_input}],
        max_tokens=200,
        temperature=0.7
    )
    
    print("\n=== Réponse du modèle ===")
    print(resp.choices[0].message.content)
    print("---------------------------------------------------\n")



=== Réponse du modèle ===
Salut ! Comment puis-je vous aider aujourd'hui ?
---------------------------------------------------

Fin de l'interaction.


Maintenant, tu peux saisir n’importe quel prompt, et tu verras la réponse du modèle.  
Tape `exit` pour quitter la boucle.

----

## Conclusion

Dans ce Notebook, nous avons :

- configuré la nouvelle API `openai >= 1.0.0` (avec `client.chat.completions.create(...)`),
- utilisé plusieurs techniques de **prompt engineering avancé** : 
  - zero-shot, 
  - few-shot, 
  - chain-of-thought,
  - self-refine,
  - un prompt interactif final.